# MEarl data

Generator for this dataset is taken from the project:
https://github.com/matthewearl/deep-anpr

In [9]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
from sklearn import cross_validation
from skimage import io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import tensorflow as tf
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8

DATA_PATH = '../data/mearl/'

## Import data

Data is stored in csv file. Images are saved as PNG files.

Data will be loaded into the following variables:
  * X - Images, (num_of_images, image_height, image_width) Tensor
  * Y - Labels, (num_of_images, 4) left, top, right, bottom 

In [2]:
def load_image(fname):
    return io.imread(DATA_PATH + fname) / 256.

df = pd.read_csv(DATA_PATH + 'train.csv')
img = load_image(df.iloc[0].image)
print('Image dimension: ' + str(img.shape))
df.head()

Image dimension: (64, 128)


,image,plate,left,top,right,bottom
0,00000000_XR30PGY.png,XR30PGY,44,28,85,38
1,00000001_PK69KBA.png,PK69KBA,70,27,116,37
2,00000002_VH93KAI.png,VH93KAI,56,26,100,37
3,00000003_AD11AAS.png,AD11AAS,43,28,85,40
4,00000004_WO98LGQ.png,WO98LGQ,83,19,123,27


Basic image information

In [3]:
xs = [load_image(fname) for fname in df['image']]
X = np.array(xs)
Y = df[['left', 'top', 'right', 'bottom']].as_matrix()
print('Images shape: ' + str(X.shape))
print('Labels shape: ' + str(Y.shape))

Images shape: (10000, 64, 128)
Labels shape: (10000, 4)


### Show sample images

In [4]:
def show_image(sub_plot, image, labels):
    rect = Rectangle((labels[0], labels[1]), labels[2]-labels[0], labels[3]-labels[1], edgecolor='r', fill=False)
    plt.imshow(image)
    gca = plt.gca()
    gca.add_patch(rect)
    
def plot_random_images(images, labels):
    plt.gray()
    fig = plt.figure()
    n = images.shape[0]
    for i in range(3):
        f = fig.add_subplot(1, 3, i+1)
        index = random.randint(0, n)
        show_image(f, images[index], labels[index])
    plt.show()    
    
plot_random_images(X, Y)    

## Build Tensorflow neural net

### Define loss function

We will optimize mean square error over all predictions.

$$MSE = \frac{1}{n} \sum_{i=1}^{n}{(\hat{y_i} - y_i)^2}$$

In [5]:
def mse(expected, predicted):
    se = tf.square(expected - predicted)
    return tf.reduce_mean(se)

Test loss function

In [23]:
xs = tf.constant(2, shape=[10, 4])
ys = tf.constant(0, shape=[10, 4])

with tf.Session() as session:
    print(session.run(mse(xs, ys)))

4


## Build simple perception

In [11]:
HIDDEN_1_SIZE = 100
PIXEL_COUNT = X.shape[1] * X.shape[2]
LABEL_COUNT = Y.shape[1]
# Create placeholders for image data and expected point positions
x_placeholder = tf.placeholder(tf.float32, shape=[None, PIXEL_COUNT])
y_placeholder = tf.placeholder(tf.float32, shape=[None, LABEL_COUNT])

# Build neural network
def build_model(xs):
    W1 = tf.Variable(tf.truncated_normal([PIXEL_COUNT, HIDDEN_1_SIZE], stddev=0.1))
    b1 = tf.Variable(tf.constant(0.1, shape=[HIDDEN_1_SIZE]))
    hidden1 = tf.nn.relu(tf.matmul(xs, W1) + b1)
    W2 = tf.Variable(tf.truncated_normal([HIDDEN_1_SIZE, LABEL_COUNT], stddev=0.1))
    b2 = tf.Variable(tf.constant(0.1, shape=[LABEL_COUNT]))
    return tf.matmul(hidden1, W2) + b2

# Define loss function
model = build_model(x_placeholder)
loss = mse(y_placeholder, model)

X2 = np.reshape(X, (X.shape[0], X.shape[1]*X.shape[2]))
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X2, Y, test_size=0.8)
MAX_ITER = 100
with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    for i in range(MAX_ITER):
        train_step.run(feed_dict={x_placeholder: X_train, y_placeholder: y_train})
        if (i+1) % (MAX_ITER/10) == 0:
            score = loss.eval(feed_dict={x_placeholder: X_test, y_placeholder: y_test})
            print('Epoch: %d, loss=%f' % (i+1, score))
    predictions = model.eval(feed_dict={x_placeholder: X2})
predictions

Epoch: 10, loss=9040595968.000000
Epoch: 20, loss=28672434.000000
Epoch: 30, loss=91163.484375
Epoch: 40, loss=404.111755
Epoch: 50, loss=109.397583
Epoch: 60, loss=108.074448
Epoch: 70, loss=108.048080
Epoch: 80, loss=108.046341
Epoch: 90, loss=108.046547
Epoch: 100, loss=108.046532


array([[ 40.01799393,  26.19499779,  86.544487  ,  37.68749237],
       [ 40.01799393,  26.19499779,  86.544487  ,  37.68749237],
       [ 40.01799393,  26.19499779,  86.544487  ,  37.68749237],
       ..., 
       [ 40.01799393,  26.19499779,  86.544487  ,  37.68749237],
       [ 40.01799393,  26.19499779,  86.544487  ,  37.68749237],
       [ 40.01799393,  26.19499779,  86.544487  ,  37.68749237]], dtype=float32)

### Show some sample predictions

In [12]:
plot_random_images(X, predictions)